In [1]:
import numpy as np
import yaml
import torch
import pickle

Load created data_points.pkl  that is created in the outputs folder when running NeuroLS over the instance file in data/JSSP/jsspNxM

In [2]:
import pickle
with open("data/dt_datasets/50x20.pkl", "rb") as handle:
    data = pickle.load(handle)
#data containsevery ls step per instance (List[List[Dict]]) -> Dict contains datapoint for one lsstep
print(data[0])

[{'step_number': 0, 'observation': tensor([ 4.9485e+01, -5.3602e+00, -5.3675e-01, -5.9402e+00,  1.2053e+00,
         4.3081e+00,  1.3114e+01,  4.8774e+00, -3.6223e+00, -6.0685e+00,
        -7.3185e+00,  1.1729e+02, -5.8553e-01, -1.0441e+00, -2.2191e+02,
        -1.3587e+00,  3.4004e+00,  1.0857e+00, -1.6428e+01,  1.6989e+00,
        -5.6771e+00, -4.8022e-01, -1.4358e+01, -3.2670e+00,  1.1141e+01,
        -1.7588e+01, -3.2501e-01,  2.4529e+00,  9.5567e+00, -1.4020e+02,
        -7.1804e+00,  4.7678e+00,  1.6725e+02,  1.3124e+00, -5.9219e+00,
        -1.1713e+00, -6.1758e+00, -8.8919e+00, -6.9958e+00,  1.1224e+01,
         1.1789e+01,  9.3029e-01,  6.0903e+00,  1.0687e+01,  6.4728e+00,
         8.6724e+00,  1.7129e+01, -2.6894e+00,  6.9243e-02,  2.2998e+00,
         9.8880e+00,  4.2317e+00,  5.7392e+00,  7.4954e+00, -1.4878e+00,
        -1.6540e+02,  1.0518e+01,  9.3499e+00,  3.9784e+00,  1.1354e+01,
         8.7155e+00,  2.1234e+00, -4.8594e+00, -5.5083e+01,  1.7305e+00,
         1.1533e

In [16]:
def calculate_returns_to_go(instance_datapoints): #iterates over al ls steps per instance and calcs rtgs
    return_to_go = 0
    return_to_go_list = []
    for datapoint in reversed(instance_datapoints): #Iterate reversed because last rtg is zero and first rtg is sum of all rewards achieved
            return_to_go = return_to_go + datapoint.get("reward")
            datapoint['returns_to_go'] = return_to_go
            return_to_go_list.append(return_to_go)
    return_to_go_list.reverse()
    return return_to_go_list

for instance in data:
    calculate_returns_to_go(instance)

[{'step_number': 51100,
  'observation': tensor([ 5.3254e+01, -7.9113e+00, -9.2592e-02, -4.9046e+00, -3.0288e-02,
           3.5004e+00,  1.0866e+01,  2.9298e+00, -1.4529e+00, -6.1052e+00,
          -6.6738e+00,  1.1987e+02,  1.2835e+00,  9.1958e-01, -2.2422e+02,
           1.2199e+00,  4.1230e+00,  3.7912e+00, -1.6804e+01,  2.6685e+00,
          -8.7804e+00,  2.1907e+00, -1.4418e+01, -5.3821e+00,  1.2613e+01,
          -1.7864e+01, -2.2831e+00,  2.0477e+00,  1.0145e+01, -1.3919e+02,
          -8.1314e+00,  4.2457e+00,  1.7414e+02,  2.2769e+00, -4.9319e+00,
          -5.6510e-02, -7.6515e+00, -1.0337e+01, -9.2194e+00,  9.8870e+00,
           1.3393e+01,  3.2599e-01,  8.2159e+00,  1.2402e+01,  8.3810e+00,
           8.2075e+00,  1.9842e+01, -1.8245e+00, -3.2512e-01,  1.0768e+00,
           8.8582e+00,  3.0843e+00,  4.6371e+00,  5.0325e+00, -1.2977e+00,
          -1.6091e+02,  1.0953e+01,  1.1468e+01,  5.4278e+00,  1.2197e+01,
           8.1887e+00,  1.3838e+00, -3.1477e+00, -5.6317e+01,

In [17]:
flattened_list = []
num_iterations = 100 #200
#For the StateActionReturnDataset during training we want a list[Dict] of single datapoints
for i in range(len(data)):
    for d in data[i]:
        d["step_number"] = d["step_number"] - num_iterations*i # This will fix the step number (Every num_iterations a new instance starts with step=0)
        d["observation"] = d["observation"].numpy()
    flattened_list.extend(data[i])


In [22]:
#Save the ready to train dataset
with open('data/dt_datasets/50x20_100it_1000inst_flat.pkl', 'wb') as f:
    pickle.dump(flattened_list, f)